In these notes, we consider a simple mixed integer linear program (MILP) and derive a Benders decomposition scheme to solve it.

In [ ]:
using Pkg; Pkg.activate(dirname(@__DIR__))
using JuMP
using HiGHS

In [ ]:
f = [1, 4];
c = [2, 3];
e = [-2; -3];
A = [1 -3; -1 -3];
E = [1 -2; -1 -1];

The full monolithic model is as follows:

In [ ]:
monolithic_model = Model(HiGHS.Optimizer);
@variable(monolithic_model, x[1:2] >= 0, Int);
@variable(monolithic_model, y[1:2] >= 0);
@constraint(monolithic_model, A*x + E*y .<= e);
@objective(monolithic_model,Min, f'*x + c'*y);
latex_formulation(monolithic_model)

In [ ]:
optimize!(monolithic_model)

In [ ]:
value.(monolithic_model[:x])

Because we are considering a small example problem, the solver was able to quickly compute a solution. However, real-life mixed integer linear programs involve tens of thousands of integer decisions, pushing even the best commercial solvers to their computational limits. 

Note that if we fix the values of integer variables $x_1$ and $x_2$, we obtain a continuous linear program which can be easily solved. For this reason, Benders decomposition is often implemented to solve MILPs, considering integer decisions as complicating variables. 

Let's initialize the master (or planning) problem:

In [ ]:
master = Model(HiGHS.Optimizer);
@variable(master, x[1:2] >= 0, Int);
@variable(master,θ>=-1000) #### Initial lower bound on the operational cost (if operational cost is always positive, this can be set to 0)
@objective(master,Min, f'*x + θ);

latex_formulation(master)

Then, the subproblem is defined including auxiliary variables $z$ that act as local copies of the master variables:

In [ ]:
subprob = Model(HiGHS.Optimizer);
@variable(subprob, z[1:2] >= 0);
@variable(subprob, y[1:2] >= 0);
@constraint(subprob,A*z + E*y .<= e);
@objective(subprob, Min, c'*y);
latex_formulation(subprob)

The Benders decomposition algorithm, starts from solving the master problem:

In [ ]:
optimize!(master)

Using the solution of the master, we define a lower bound to the optimal value of our original MILP:

In [ ]:
LB = objective_value(master)

and also guesses for the master variables:

In [ ]:
xk = value.(master[:x])

In [ ]:
xk_history = [xk]

We fix these guesses in the subproblem adding the constraints:

In [ ]:
fix.(subprob[:z],xk;force=true)
FixRef.(subprob[:z])

And solve it:

In [ ]:
optimize!(subprob)

We can now compute an upper bound to the optimal value of the original MILP as: 

In [ ]:
UB = f'*xk + objective_value(subprob)

With lower and upper bounds, we can compute the optimality gap, which gives a conserative estimate of the degree of sub-optimality of our current best guess:

In [ ]:
gap = (UB-LB)/abs(UB)

To improve our guesses, we need to include additional information into the master problem. 

Hence, we derive the so called optimality cuts, which are based on the dual solutions associated with the constraints fixing the values of the master variables in the subproblems:

In [ ]:
λ = dual.(FixRef.(subprob[:z]))

Having only one subproblem, we add one optimality cut per iteration to the master problem, which is given by:

$$\theta \geq f_k + \lambda^T(x - x_k)$$

where $f_k$ is the objective value of the subproblem obtained fixing variables $z$ to the value $x_k$.

In JuMP, this constraint is implemented as:

In [ ]:
@constraint(master, master[:θ] >= objective_value(subprob) + λ'*(master[:x]-xk))

In [ ]:
cut_history = [[objective_value(subprob);λ[1];λ[2]]];

We now repeat the step above until a zero optimality gap is reached:

Solve updated master problem:

In [ ]:
optimize!(master)

And obtain new guesses and lower bound:

In [ ]:
xk = value.(master[:x])

In [ ]:
push!(xk_history,xk)

In [ ]:
LB = objective_value(master)

We fix the master variable values into the subproblem:

In [ ]:
fix.(subprob[:z],xk;force=true)
FixRef.(subprob[:z])

and solve the subproblem to obtain an upper bound:

In [ ]:
optimize!(subprob)

In [ ]:
UB = min(UB,f'*xk + objective_value(subprob))

The current optimality gap is:

In [ ]:
gap = (UB-LB)/abs(UB)

We have reduced the optimality gap!

Next, we compute a new optimality cut to be added to the master:

In [ ]:
λ = dual.(FixRef.(subprob[:z]))

In [ ]:
cuts_history = push!(cut_history,[objective_value(subprob);λ[1];λ[2]])

In [ ]:
@constraint(master, master[:θ] >= objective_value(subprob) + λ'*(master[:x]-xk))

We solve updated master problem:

In [ ]:
optimize!(master)

And obtain new guesses and lower bound:

In [ ]:
xk = value.(master[:x])

In [ ]:
push!(xk_history,xk)

In [ ]:
LB = objective_value(master)

We fix the master variable values into the subproblem:

In [ ]:
fix.(subprob[:z],xk;force=true)
FixRef.(subprob[:z])

and solve the subproblem to obtain an upper bound:

In [ ]:
optimize!(subprob)

In [ ]:
UB = min(UB,f'*xk + objective_value(subprob))

The current optimality gap is:

In [ ]:
gap = (UB-LB)/abs(UB)

Next, we compute a new optimality cut to be added to the master:

In [ ]:
λ = dual.(FixRef.(subprob[:z]))

In [ ]:
cuts_history = push!(cut_history,[objective_value(subprob);λ[1];λ[2]]);

In [ ]:
@constraint(master, master[:θ] >= objective_value(subprob) + λ'*(master[:x]-xk))

We solve updated master problem:

In [ ]:
optimize!(master)

And obtain new guesses and lower bound:

In [ ]:
xk = value.(master[:x])

In [ ]:
push!(xk_history,xk)

In [ ]:
LB = objective_value(master)

We fix the master variable values into the subproblem:

In [ ]:
fix.(subprob[:z],xk;force=true)
FixRef.(subprob[:z])

and solve the subproblem to obtain an upper bound:

In [ ]:
optimize!(subprob)

In [ ]:
UB = min(UB,f'*xk + objective_value(subprob))

The current optimality gap is:

In [ ]:
gap = (UB-LB)/abs(UB)

Zero optimality gap! We have converged to a solution of our MILP, which coincides with the one obtained solving the monolithic model:

In [ ]:
xk

In [ ]:
value.(monolithic_model[:x])

Note that in general the two solutions need not be the same, but they will correspond to the same objective function value.

In [ ]:
cut_history

In [ ]:
xk_history

In [ ]:
function total_cost(x1,x2)
    xv = [x1;x2];
    fix.(subprob[:z],xv;force=true)
    set_silent(subprob)
    optimize!(subprob)
    return f'*xv + objective_value(subprob)
end

In [ ]:
using Plots

In [ ]:
pythonplot()

In [ ]:
x1range = range(0,520)
x2range = range(0,520)

In [ ]:
plot(x1range, x2range, total_cost, 
    st = :surface,  # Surface plot type
    xlabel = "x1",
    ylabel = "x2",
    zlabel = "Objective value",
    color = :viridis,  # Colormap for the surface
    alpha = 0.7,  # Transparency of the surface
    legend = false
)

In [ ]:
plot!(x1range, x2range, (x, y) -> cut_history[1][1] + cut_history[1][2]*(x - xk_history[1][1]) + cut_history[1][3]*(y - xk_history[1][2]),
    st = :surface, 
    color = :red, 
    alpha = 0.3
)